## Εισαγωγή βιβλιοθηκών

In [1]:
from pykml import parser
from lxml import etree
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import os
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

##  Συνάρτηση για το κατέβασμα των KML από το https://sentinel.esa.int/web/sentinel/copernicus/sentinel-2/acquisition-plans

In [3]:
def download_kml():
    KML_url = 'https://sentinel.esa.int/web/sentinel/copernicus/sentinel-2/acquisition-plans'
    base_url = 'https://sentinel.esa.int/'
    response = requests.get(KML_url)
    sentinel_types = ['sentinel-2a', 'sentinel-2b']

    if response.status_code == 200:
        print('Response OK!')

        # Parse the HTML content
        html_content = BeautifulSoup(response.content, 'html.parser')
        
        for sentinel_type in sentinel_types:
            elements = html_content.find_all(class_=sentinel_type)

            if elements:
                for element in elements:
                    # Find all links within the element
                    links = element.find_all('a')

                    for link in links:
                        if 'href' in link.attrs:
                            file_url = link['href']
                            link_text = link.get_text(strip=True)
                            print(f'Downloading {link_text} from {base_url + file_url}')

                            # Construct the file name and full URL
                            file_name = os.path.basename(file_url) + '.kml'
                            full_file_url = base_url + file_url
                            
                            # Download the file
                            urlretrieve(full_file_url, file_name)
                            print(f'File downloaded as {file_name}\n')
                        else:
                            print('No href found for this link.')
            else:
                print(f'No elements found for {sentinel_type}')
    else:
        print('Response not OK!!!')

    return 0

In [4]:
download_kml()

Response OK!
File downloaded as s2a_mp_acq__kml_20240822t120000_20240909t150000.kml

File downloaded as s2a_mp_acq__kml_20240808t090000_20240826t120000.kml

File downloaded as s2b_mp_acq__kml_20240815t120000_20240902t150000.kml

File downloaded as s2b_mp_acq__kml_20240803t120000_20240819t150000.kml



0

## Συνάρτηση εξαγωγής δεδομένων απο KML σε GeoDataFrame

In [5]:
# Function to recursively extract placemarks and save to GDB
def extract_placemarks_to_gdb(folder, gdf_data=[]):
    if hasattr(folder, 'Placemark'):
        for placemark in folder.Placemark:
            data = {
                'name': placemark.name.text if hasattr(placemark, 'name') else "No Name",
                'geometry': coordinates_to_polygon(
                    placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text
                ) if hasattr(placemark, 'Polygon') else None,
                'styleUrl': placemark.styleUrl.text if hasattr(placemark, 'styleUrl') else None,
                'visibility': int(placemark.visibility.text) if hasattr(placemark, 'visibility') else None,
                'begin': placemark.TimeSpan.begin.text if hasattr(placemark, 'TimeSpan') else None,
                'end': placemark.TimeSpan.end.text if hasattr(placemark, 'TimeSpan') else None,
            }

            # Extract ExtendedData fields
            if hasattr(placemark, 'ExtendedData'):
                for data_field in placemark.ExtendedData.Data:
                    data_name = data_field.get('name')
                    data_value = data_field.value.text
                    data[data_name] = data_value

            gdf_data.append(data)
            
    if hasattr(folder, 'Folder'):
        for subfolder in folder.Folder:
            extract_placemarks_to_gdb(subfolder, gdf_data)
            
    return gdf_data

## Συνάρτηση μετατροπής συντεταγμένων σε πολύγωνο

In [6]:
# Function to convert coordinates to a Polygon object
def coordinates_to_polygon(coordinates):
    # Coordinates are typically in the form "lon,lat,alt lon,lat,alt ..."
    points = []
    for coord in coordinates.split():
        lon, lat, _ = map(float, coord.split(","))
        points.append((lon, lat))
    return Polygon(points)

## Main

In [12]:
# Main script
filename = 'D:\\praktiki_noa\\praktiki_scripts\\sentinel_acquisition_plans\\s2a_mp_acq__kml_20240822t120000_20240909t150000.kml'
with open(filename) as f:
    root = parser.parse(f).getroot()

# Extract all placemarks and their geometries
gdf_data = extract_placemarks_to_gdb(root.Document)
print(len(gdf_data))

# Convert to GeoDataFrame
# Specify columns including dynamic fields extracted from ExtendedData
columns = ['name', 'geometry', 'styleUrl', 'visibility', 'begin', 'end'] + list({key for row in gdf_data for key in row.keys() if key not in ['name', 'geometry', 'styleUrl', 'visibility', 'begin', 'end']})

# Create the GeoDataFrame
gdf = gpd.GeoDataFrame(gdf_data, columns=columns)

# Print all placemarks' names and coordinates
print("Placemarks: \n", len(gdf))

print(gdf)

# Διαγραφή των διπλότυπων εγγραφών, κρατώντας μόνο την πρώτη εμφάνιση
# Εδώ χρησιμοποιούμε τη στήλη 'name' ως κριτήριο, μπορείς να προσθέσεις περισσότερες στήλες αν χρειάζεται
gdf_unique = gdf.drop_duplicates(subset= gdf.columns)

# Εμφάνιση του GeoDataFrame μετά την αφαίρεση των διπλότυπων
print("\n GeoDataFrame μετά την αφαίρεση των διπλότυπων:")
print(gdf_unique)
print("o kodikas exei treksei", len(gdf)/len(gdf_unique), "fores \n")

# Check and print the CRS of the GeoDataFrame
gdf_unique.set_crs('EPSG:4326', inplace=True)
print("Coordinate Reference System (CRS):")
print(gdf_unique.crs)

# # Define the output GDB path
# gdb_path = "output.gdb"
# layer_name = "placemarks"

# # Check if the GDB already exists; if not, create the directory
# if not os.path.exists(gdb_path):
#     os.mkdir(gdb_path)

# # Save to the GDB
# gdf.to_file(gdb_path, layer=layer_name, driver="OpenFileGDB")


4185
Placemarks: 
 4185
         name                                           geometry  \
0     47882-5  POLYGON ((-25.76209 -12.86507, -27.06175 -12.5...   
1     47884-4  POLYGON ((-67.72001 24.06491, -69.10468 24.350...   
2     47885-2  POLYGON ((-106.83636 -34.75044, -108.38159 -34...   
3     47892-3  POLYGON ((79.25366 -26.31507, 77.83752 -26.011...   
4     47896-2  POLYGON ((-18.21169 -12.87404, -19.51139 -12.5...   
...       ...                                                ...   
4180  47997-6  POLYGON ((-49.10947 -43.11359, -50.84525 -42.7...   
4181  47997-7  POLYGON ((-50.07194 -45.72454, -51.88449 -45.3...   
4182  47979-4  POLYGON ((-130.80084 -28.45875, -129.35673 -28...   
4183  47990-1  POLYGON ((-34.58158 23.44241, -33.20353 23.727...   
4184  48133-1  POLYGON ((-80.60558 78.17766, -75.87767 79.095...   

          styleUrl  visibility                    begin  \
0          S2A_VIC           0  2024-08-22T12:05:15.708   
1          S2A_VIC           0  2024-08-2

d:\anaconda3\envs\env_praktiki_noa\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Συνάρτηση εύρεσης περάσματος δορυφόρου σε ένα συγκεκριμένο σημείο

In [7]:
def get_observation_times(gdf, latitude, longitude):
    # Create a Point object for the given latitude and longitude
    point = Point(longitude, latitude)

    # List to hold observation times for all matching polygons
    observation_info = []

    # Ensure the CRS is set for the GeoDataFrame
    if gdf.crs is None:
        raise ValueError("CRS is not set for the GeoDataFrame.")
    
    # If the CRS is not EPSG:4326, transform the point
    if gdf.crs.to_string() != 'EPSG:4326':
        point = gpd.GeoSeries([point], crs='EPSG:4326').to_crs(gdf.crs).iloc[0]

    # Search for all polygons that contain the point
    for i, row in gdf.iterrows():
        if row['geometry'] and row['geometry'].contains(point):
            # Append the ObservationTimeStart and ObservationTimeStop to the list
            observation_info.append({
                'Id': row.get('ID'),
                'ObservationTimeStart': row.get('ObservationTimeStart'),
                'ObservationTimeStop': row.get('ObservationTimeStop')
            })

    # If no polygon contains the point, return an empty list
    return observation_info


In [14]:
latitude = 40.89  # Replace with the latitude (φ) of the point you want to check
longitude = 24.05  # Replace with the longitude (λ) of the point you want to check


observation_info = get_observation_times(gdf_unique, latitude, longitude)
"""
if observation_info:
    print("There are ", len(observation_info), " polygons containing the point.")
    print("Observation info for the point:", observation_info)
"""
if observation_info:
    print("There are", len(observation_info), "polygons containing the point.")
    print("Observation info for the point:")
    for i, info in enumerate(observation_info, 1):
        print(f"{i}. {info}")
else:    
    print("The point is not within any of the polygons.")

There are 4 polygons containing the point.
Observation info for the point:
1. {'Id': '47952-1', 'ObservationTimeStart': '2024-08-27T09:05:50.394', 'ObservationTimeStop': '2024-08-27T09:35:58.002'}
2. {'Id': '47995-2', 'ObservationTimeStart': '2024-08-30T09:20:24.973', 'ObservationTimeStop': '2024-08-30T09:42:32.717'}
3. {'Id': '48095-1', 'ObservationTimeStart': '2024-09-06T09:05:50.394', 'ObservationTimeStop': '2024-09-06T09:35:58.002'}
4. {'Id': '48138-2', 'ObservationTimeStart': '2024-09-09T09:20:24.973', 'ObservationTimeStop': '2024-09-09T09:42:32.717'}
